<a href="https://colab.research.google.com/github/ARAN1218/RealEstateRentPrediction_AI/blob/main/jupyter%20notebook/SUUMO%E3%82%B9%E3%82%AF%E3%83%AC%E3%82%A4%E3%83%94%E3%83%B3%E3%82%B0_%E6%94%B9%E5%96%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#必要なライブラリをインポート
from bs4 import BeautifulSoup
import requests
from time import sleep
import json
import pandas as pd
from tqdm.notebook import tqdm


#後でformatでページ数を代入するので、urlの内「pn=」の部分は「pn={}」としておく
urls = {
    #さがキャン
    '相模原':"https://suumo.jp/jj/common/ichiran/JJ901FC004/?ar=030&ta=14&sc=14151&sc=14152&sc=14153&kwd=&cb=0.0&ct=9999999&kb=0&kt=9999999&km=1&xb=0&xt=9999999&et=9999999&cn=9999999&newflg=0&pn={}",
    '町田':"https://suumo.jp/jj/common/ichiran/JJ901FC004/?ar=030&ta=13&sc=13209&kwd=&cb=0.0&ct=9999999&kb=0&kt=9999999&km=1&xb=0&xt=9999999&et=9999999&cn=9999999&newflg=0&pn={}",
    '横浜':"https://suumo.jp/jj/common/ichiran/JJ901FC004/?initFlg=1&seniFlg=1&pc=30&ar=030&ta=14&sa=01&newflg=0&km=1&bs=040&pn={}",
    
    #青キャン
    '渋谷':"https://suumo.jp/jj/common/ichiran/JJ901FC004/?ar=030&ta=13&sc=13113&kwd=&cb=0.0&ct=9999999&kb=0&kt=9999999&km=1&xb=0&xt=9999999&et=9999999&cn=9999999&newflg=0&pn={}",
    '新宿':"https://suumo.jp/jj/common/ichiran/JJ901FC004/?initFlg=1&seniFlg=1&pc=30&ar=030&ta=13&scTmp=13104&kb=0&xb=0&newflg=0&km=1&sc=13104&bs=040&pn={}",
    '港':"https://suumo.jp/jj/common/ichiran/JJ901FC004/?ar=030&ta=13&sc=13103&kwd=&cb=0.0&ct=9999999&kb=0&kt=9999999&km=1&xb=0&xt=9999999&et=9999999&cn=9999999&newflg=0&pn={}",
    '品川':"https://suumo.jp/jj/common/ichiran/JJ901FC004/?ar=030&ta=13&sc=13109&kwd=&cb=0.0&ct=9999999&kb=0&kt=9999999&km=1&xb=0&xt=9999999&et=9999999&cn=9999999&newflg=0&pn={}",
    '目黒':"https://suumo.jp/jj/common/ichiran/JJ901FC004/?ar=030&ta=13&sc=13110&kwd=&cb=0.0&ct=9999999&kb=0&kt=9999999&km=1&xb=0&xt=9999999&et=9999999&cn=9999999&newflg=0&pn={}",
    '世田谷':"https://suumo.jp/jj/common/ichiran/JJ901FC004/?ar=030&ta=13&sc=13112&kwd=&cb=0.0&ct=9999999&kb=0&kt=9999999&km=1&xb=0&xt=9999999&et=9999999&cn=9999999&newflg=0&pn={}",
    '杉並':"https://suumo.jp/jj/common/ichiran/JJ901FC004/?ar=030&ta=13&sc=13115&kwd=&cb=0.0&ct=9999999&kb=0&kt=9999999&km=1&xb=0&xt=9999999&et=9999999&cn=9999999&newflg=0&pn={}",
    '中野':"https://suumo.jp/jj/common/ichiran/JJ901FC004/?ar=030&ta=13&sc=13114&kwd=&cb=0.0&ct=9999999&kb=0&kt=9999999&km=1&xb=0&xt=9999999&et=9999999&cn=9999999&newflg=0&pn={}"
}


#不動産データスクレイピングのメインプログラム
#Real_Estate_Data_Scraping
def REDS(start,end,place,pre_results=[]):
    #初期値
    d_list = pre_results
    url = urls[place]
    
    #pre_resultsにデータが入っていた場合、進捗管理リストのprogressにページの番号を入れておく。
    if len(d_list) > 0:
        progress = list(pd.DataFrame(d_list)['ページ'].unique())
    else:
        progress = []
    
    #range(start,end+1,2)とすれば、2の部分を消せば指定した全ページのデータが取得できる。
    for i in tqdm(range(start,end+1)):
        
        if i in progress:
            continue
        
        #途中でエラーが発生してもそれまでの結果が保存できるようにする。
        try:
            
            #ページを遷移させる
            target_url = url.format(i)
            
            #bot対策の対策として、ヘッダーを偽装する
            headers = {
                "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15"
            }

            #Requestsを用いてtarget_urlにアクセスする
            r = requests.get(target_url, headers=headers)

            #サーバー負荷軽減の為、ループ毎に1秒間隔を空ける
            sleep(1)

            #取得したHTMLをBeautifulSoupで解析する
            soup = BeautifulSoup(r.text)

            #BeautifulSoupで解析したHTMLの内、欲しい不動産情報が乗っている部分を取得
            contents = soup.find_all('div',class_='cassettebox js-normalLink js-cassetLink')

            #1ページ当たり30件の不動産データが表示されていれば、リストcontentsは30個の要素を持つはずなので、それらを一つずつ取り出してデータを取得する。
            for content in contents:

                #再開機能の為、取得元のページ数を保存しておく
                pages = i

                #上の行のデータを取得
                rows = content.find_all('table',class_='listtable')
                address = rows[0].find_all('div',class_='infodatabox-box-txt')[0].text
                station = rows[0].find_all('div',class_='infodatabox-box-txt')[1].text
                access = rows[0].find_all('div',class_='infodatabox-box-txt')[2].text

                #下の行のデータを取得
                r_fees = rows[1].find_all('dd',class_='infodatabox-details-txt')[0].text[:-2]
                mc_fees = rows[1].find_all('dd',class_='infodatabox-details-txt')[1].text[:-1]
                k_fees = rows[1].find_all('dd',class_='infodatabox-details-txt')[2].text.split('/')[0]
                s_fees = rows[1].find_all('dd',class_='infodatabox-details-txt')[2].text.split('/')[1][:-2]
                area = rows[1].find_all('dd',class_='infodatabox-details-txt')[3].text[:-2]
                layout = rows[1].find_all('dd',class_='infodatabox-details-txt')[4].text
                age = rows[1].find_all('div',class_='infodatabox-box-txt')[2].text

                #取得した各種データを辞書dに格納する
                d = {
                'ページ':pages,
                '住所':address,
                '路線':station,
                '交通':access,
                '賃料':r_fees,
                '管理共益費':mc_fees,
                '礼金':k_fees,
                '敷金':s_fees,
                '専有面積':area,
                '間取り':layout,
                '築年数':age
                }

                #辞書dのデータをリストd_listに格納する
                d_list.append(d)

                #重複したデータを削除する
                #d_list = list(map(json.loads,set(map(json.dumps,d_list))))

            #進捗を報告させる
            print("d_list's progress:",i,"page　　",len(d_list))
            print(target_url)
            
        #リストにある存在しないページにアクセスした場合、そのページの読み込みをスキップする。
        except IndexError:
            continue
            
        #スクレイピングが中断されても、その時点までに読み込んだデータを出力できるようにする。
        except:
            break
    
    #スクレイピングが終わった事を通知
    print('Scraping Completed!')
    return d_list


In [ ]:
#進捗リセット
reds_pre_results = []

In [ ]:
#スクレイピングに必要なパラメータを入力
start = 1  #初めのページ数
end = 613  #終わりのページ数
place = '町田'  #(辞書urlsに入っている内の)読み込む地域

reds_test = REDS(start,end,place,reds_pre_results)
reds_pre_results = reds_test
pd.DataFrame(reds_pre_results)

In [ ]:
#読み込んだページ数
pd.DataFrame(reds_test)['ページ'].max()

print(len(reds_pre_results))

In [ ]:
reds_df = pd.DataFrame(reds_test)
#display(reds_dumped_df.head(10))
#reds_dumped_df = reds_df.drop_duplicates()
print(len(reds_df))
#print(len(reds_dumped_df))
#reds_dumped_df.drop(['ページ'],axis=1,inplace=True)
#reds_dumped_df.reset_index(drop=True,inplace=True)
#print(len(reds_dumped_df))
display(reds_df.head(10))

#reds_dumped_df['交通'].unique()
#reds_df.to_pickle('SUUMO_ _Bigdata.pickle')

print(len(reds_pre_results))
print(len(reds_dumped_df))

In [ ]:
#相模原のデータを1200ページ集めて、リークを防ぎつつデータ量で暴力振るった。
from sklearn.model_selection import train_test_split

#reds_dumping = pd.read_pickle('SUUMO_Bigdata/SUUMO_Sagamihara_Bigdata.pickle')
#reds_dumping = pd.read_pickle('SUUMO_Bigdata/SUUMO_Machida_Bigdata.pickle')
#reds_dumping = pd.read_pickle('SUUMO_Bigdata/SUUMO_Minato_Bigdata.pickle')
#reds_dumping = pd.read_pickle('SUUMO_Bigdata/SUUMO_Shibuya_Bigdata.pickle')
#reds_dumping = pd.read_pickle('SUUMO_Bigdata/SUUMO_Yokohama_Bigdata.pickle')
#reds_dumping = pd.read_pickle('SUUMO_Bigdata/SUUMO_Shinjuku_Bigdata.pickle')
#reds_dumping = pd.read_pickle('SUUMO_Bigdata/SUUMO_Shinagawa_Bigdata.pickle')
#reds_dumping = pd.read_pickle('SUUMO_Bigdata/SUUMO_Meguro_Bigdata.pickle')
#reds_dumping = pd.read_pickle('SUUMO_Bigdata/SUUMO_Setagaya_Bigdata.pickle')
#reds_dumping = pd.read_pickle('SUUMO_Bigdata/SUUMO_Suginami_Bigdata.pickle')
#reds_dumping = pd.read_pickle('SUUMO_Bigdata/SUUMO_Nakano_Bigdata.pickle')

reds_dumping = pd.read_pickle('SUUMO_Bigdata/SUUMO_Sagakyan_Bigdata.pickle')
#reds_dumping = pd.read_pickle('SUUMO_Bigdata/SUUMO_Aokyan_Bigdata.pickle')

print("処理前データ数：",len(reds_dumping))
#display(reds_dumping.head(10))
reds_dumping.drop('ページ',axis=1,inplace=True)
reds_dumping = reds_dumping.drop_duplicates()
print("処理後データ数：",len(reds_dumping))

#reds_dumping.drop(["路線"],axis=1,inplace=True)

df,df_t = train_test_split(reds_dumping,test_size=0.25,random_state=71,shuffle=True)
print("処理後学習データ数：",len(df))
print("処理後テストデータ数：",len(df_t))
#display(df.head(10))
#df_t = reds_dumping[reds_dumping.index <= len(reds_dumping)/4]
print(sorted(df['賃料'].unique().astype(float)),"\n")
print(sorted(df_t['賃料'].unique().astype(float)))
df = df.drop_duplicates()
df_t = df_t.drop_duplicates()
df_t.head(10)

In [ ]:
#必要なライブラリをインポート
import re
import pandas as pd
from sklearn.preprocessing import LabelEncoder

#pandasのデータフレームの列表示制限を解除
#pd.set_option('display.max_rows',None)


#既にREDSを用いて変数d_listとd_t_listにスクレイピングしたデータが保存され、それぞれdf,df_testにデータフレーム型に変換されて代入されているとする
#d_list = REDS(start,end,place)
#d_t_list = REDS(start+1,end+1,place)


#オリジナル予測の為にラベルエンコーダーを関数の外で用意する
#LE1,LE2,LE3 = LabelEncoder(),LabelEncoder(),LabelEncoder()


#学習データ/テストデータを同時に前処理する関数
#Real_Estate_Data_Preprocessing
def REDP(d_df,d_t_df):
    #バラバラになっているインデックスを振り直して整理する。
    df_l = d_df.reset_index(drop=True)
    df_t = d_t_df.reset_index(drop=True)
    
    #df内の余計な文字(\n等)を消去する
    for df in [df_l,df_t]:
        df['礼金'] = df['礼金'].str.replace(r'\(.+\)','')
        for element in ['住所','交通','専有面積','間取り','築年数','礼金','敷金']:
            df[element] = df[element].str.translate(str.maketrans({'-':'','\r':'','\n':'','\t':''
                                                                   ,'バ':'','ス':'','分':'','徒':''
                                                                   ,'歩':'','停':'','―':'','築':''
                                                                   ,'年':'','新':'','万':'','(':''
                                                                   ,'円':'','車':'','以':'','上':''}))
    
    #オリジナル予測の為にラベルエンコーダーをグローバル変数として用意する
    global LE1,LE2,LE3
    LE1,LE2,LE3 = LabelEncoder(),LabelEncoder(),LabelEncoder()
    
    #ラベルエンコーダーの学習を行い、列毎に変換の数字を合わせる
    #注意:余計な文字を取り除いた段階でテストデータも一緒にラベルエンコードしないと、テストデータにのみ存在する住所等を変換できなくなってしまう為、学習データと縦に連結してfitさせる
    LE_df = pd.concat([df_l,df_t])
    LE1.fit(LE_df['住所'])
    LE2.fit(LE_df['路線'])
    LE3.fit(LE_df['間取り'])
    
    #ラベルエンコーディングを行い、文字列を数値化する
    for df in [df_l,df_t]:
        df['住所'] = LE1.transform(df['住所'])
        df['路線'] = LE2.transform(df['路線'])
        df['間取り'] = LE3.transform(df['間取り'])
    
        #欠損値を0に統一する
        df.replace('',0,inplace=True)
    
    #データ型をfloat64で統一する
    #astypeの使用上、for文に組み込めなかった
    df_l = df_l.astype('float64')
    df_t = df_t.astype('float64')
    
    #敷金・礼金の特徴量を「賃料のnヶ月分」として表す(そうしないとリークになってしまうから)
    for name in ['敷金','礼金']:
        for df in [df_l,df_t]:
            df[name] = df[name] / df['賃料']
    
    return df_l,df_t


#REDP(前処理したいリスト学習データ,前処理したいリストテストデータ)
df_l,df_t = REDP(df,df_t)

#確認用
display(df_l.head(10))
display(df_t.head(10))


#オリジナル予測の為にラベルエンコーダーの学習した要素を変数に格納しておく
adressC,stationC,layoutC = LE1.classes_, LE2.classes_ ,LE3.classes_
print(adressC,'\n','\n',stationC,'\n','\n',layoutC)



In [ ]:
del df_l['路線']
del df_t['路線']

print(df_l.head(1))
print(df_t.head(1))

In [ ]:
#敷金・礼金の調整
print(df_l['敷金'].unique())
print(df_l['礼金'].unique())

In [ ]:
#XGBOOSTのモデルのクラス
import xgboost as xgb


class Model:

    def __init__(self, params=None):
        self.model = None
        if params is None:
            self.params = {}
        else:
            self.params = params

    def fit(self, tr_x, tr_y, va_x, va_y):
        params = {
            #'booster': 'gbtree',
            'objective': 'reg:squarederror',
            'eta': 0.1,
            'gamma': 0.0,
            'alpha': 0.0,
            'lambda': 1.0,
            'min_child_weight': 1,
            'max_depth': 5,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'random_state': 71,
        }
        params.update(self.params)
        num_round = 20
        dtrain = xgb.DMatrix(tr_x, label=tr_y)
        dvalid = xgb.DMatrix(va_x, label=va_y)
        watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
        self.model = xgb.train(params, dtrain, num_round, early_stopping_rounds=10, evals=watchlist)

    def predict(self, x):
        data = xgb.DMatrix(x)
        pred = self.model.predict(data)
        return pred


In [ ]:
#必要なライブラリのインポート
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import xgboost as xgb

# Modelクラスを定義しているものとする
# Modelクラスは、fitで学習し、predictで予測値の確率を出力する

scores = []
models = []

train_x = df_l.drop(['賃料'], axis=1)
train_y = df_l['賃料']

# KFoldクラスを用いてクロスバリデーションの分割を行う
kf = KFold(n_splits=10, shuffle=True, random_state=71)
for tr_idx, va_idx in kf.split(train_x):
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    # 学習の実行、バリデーションデータの予測値の出力、スコアの計算を行う
    model = Model()
    model.fit(tr_x, tr_y, va_x, va_y)
    va_pred = model.predict(va_x)
    score = np.sqrt(mean_squared_error(va_y,va_pred))
    scores.append(score)
    models.append(model)

# 各foldのスコアの平均をとる
print(np.mean(scores))
print(models)

In [ ]:
#4つのモデルを用いてアンサンブル予測

test_x = df_t.drop(['賃料'], axis=1)
test_y = df_t['賃料']
pred_list = 0

for model in models:
    pred = model.predict(test_x)
    pred_list += pred
final_pred = pred_list / len(models)
display(pd.DataFrame({'正解':test_y, 
                      '予測':final_pred}))

print('MAE:',mean_absolute_error(test_y,final_pred))
print('MSE:',mean_squared_error(test_y,final_pred))
print('RMSE:',np.sqrt(mean_squared_error(test_y,final_pred)))

In [ ]:
#必要なライブラリをインポート
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import mean_squared_error


#hyperoptを使ったパラメータ探索
def score(params):
    # パラメータを与えたときに最小化する評価指標を指定する
    # 具体的には、モデルにパラメータを指定して学習・予測させた場合のスコアを返すようにする

    # max_depthの型を整数型に修正する
    params['max_depth'] = int(params['max_depth'])

    # Modelクラスを定義しているものとする
    # Modelクラスは、fitで学習し、predictで予測値の確率を出力する
    model = Model(params)
    model.fit(tr_x, tr_y, va_x, va_y)
    va_pred = model.predict(va_x)
    score = np.sqrt(mean_squared_error(va_y, va_pred))
    print(f'params: {params}, rmse: {score:.4f}')

    # 情報を記録しておく
    history.append((params, score))

    return {'loss': score, 'status': STATUS_OK}


#データ毎にハイパーパラメータチューニングをhyperoptで行う。
#HyperPramaterTuning_XGBOOST
def HPT_XGB(df_l):
    train_x = df_l.drop(['賃料'], axis=1)
    train_y = df_l['賃料']

    # 学習データを学習データとバリデーションデータに分ける
    kf = KFold(n_splits=4, shuffle=True, random_state=71)
    tr_idx, va_idx = list(kf.split(train_x))[0]
    global tr_x,va_x,tr_y,va_y
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    # hp.choiceでは、複数の選択肢から選ぶ
    # hp.uniformでは、下限・上限を指定した一様分布から抽出する。引数は下限・上限
    # hp.quniformでは、下限・上限を指定した一様分布のうち一定の間隔ごとの点から抽出する。引数は下限・上限・間隔
    # hp.loguniformでは、下限・上限を指定した対数が一様分布に従う分布から抽出する。引数は下限・上限の対数をとった値

    # 探索するパラメータの空間を指定する
    param_space = {
        'max_depth': hp.quniform('max_depth', 3, 9, 1),
        'min_child_weight': hp.loguniform('min_child_weight', np.log(0.1), np.log(10)),
        'subsample': hp.quniform('subsample', 0.6, 0.95, 0.05),
        'colsample_bytree': hp.quniform('colsample_bytree', 0.6, 0.95, 0.05),
        'gamma': hp.loguniform('gamma', np.log(1e-8), np.log(1.0)),
        'alpha' : hp.loguniform('alpha', np.log(1e-8), np.log(1.0)),
        'lambda' : hp.loguniform('lambda', np.log(1e-6), np.log(10.0)),
    }

    # hyperoptによるパラメータ探索の実行
    max_evals = 100
    trials = Trials()
    global history
    history = []
    fmin(score, param_space, algo=tpe.suggest, trials=trials, max_evals=max_evals)

    # 記録した情報からパラメータとスコアを出力する
    # （trialsからも情報が取得できるが、パラメータの取得がやや行いづらいため）
    history = sorted(history, key=lambda tpl: tpl[1])
    best = history[0]
    print("\n",f'best params:{best[0]}, score:{best[1]:.4f}')

    return best[0]

best_params = HPT_XGB(df_l)
#print(f'best_params:{best_params}')

In [ ]:
#必要なライブラリのインポート
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import xgboost as xgb
import japanize_matplotlib #matplotlibでグラフを描写する際、日本語が文字化けするのを防ぐ。
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import shap


#pandasのデータフレームの列表示制限を解除
pd.set_option('display.max_rows',None)


#既にREDP関数で学習データ/テストデータの前処理が終わっているとする
#df_l,df_t = REDP(d_list,d_t_list)


#不動産賃料予測AIモデルの学習プログラム
#HPT_XGBで取得したパラメータを利用する。
#Real_Estate_Rent_Learning
def RERL(df_l,best_params):
    #データを説明変数と目的変数に分ける
    target = df_l['賃料']
    train = df_l.drop(['賃料'],axis=1)
    
    #ホールドアウト法で学習データとテストデータに分ける
    kf = KFold(n_splits=4,shuffle=True,random_state=71)
    tr_idx,va_idx = list(kf.split(train))[0]
    tr_x,va_x = train.iloc[tr_idx],train.iloc[va_idx]
    tr_y,va_y = target.iloc[tr_idx],target.iloc[va_idx]
    
    #データ型をXGBOOST用に適合させる
    dtrain = xgb.DMatrix(tr_x,label=tr_y)
    dvalid = xgb.DMatrix(va_x,label=va_y)
    
    #XGBOOSTのモデルを作成
    num_round = 1000
    early_stopping_rounds=20
    #params = {'objective':'reg:squarederror','silent':1,'random_state':71}
    watchlist = [(dtrain,'train'),(dvalid,'eval')]
    model = xgb.train(best_params,dtrain,num_round,early_stopping_rounds=early_stopping_rounds,evals=watchlist)
    
    
    #テストデータとその予測結果を表示
    va_pred = model.predict(dvalid)
    df_true = pd.DataFrame(list(va_y),columns=['賃料'])
    df_pred = pd.DataFrame(va_pred,columns=['予測値'])
    display(pd.concat([df_true,df_pred],axis=1))
    #print(list(va_y))
    #display(list(va_pred))
    
    #モデルの性能を表示
    print('MAE:',mean_absolute_error(va_y,va_pred))
    print('MSE:',mean_squared_error(va_y,va_pred))
    print('RMSE:',np.sqrt(mean_squared_error(va_y,va_pred)))
    
    #変数の予測への寄与率をグラフにして表示
    shap.initjs()
    explainer = shap.TreeExplainer(model = model)
    shap_values = explainer.shap_values(X = tr_x)
    shap.summary_plot(shap_values, tr_x)
    shap.summary_plot(shap_values, tr_x, plot_type = "bar")
    
    return model

#RERL(学習データ)
model = RERL(df_l,best_params)


In [ ]:
#必要なライブラリをインポート
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import xgboost as xgb

#pandasのデータフレームの列表示制限を解除
pd.set_option('display.max_rows',None)


#既に学習データとREDP関数を用いてAIが作成されているものとする
#model = RERL(df_l)


#RERLで作ったAIを用いて、テストデータの不動産の賃料を予測する関数
#Real_Estate_Rent_Divided_Prediction
def RERDP(df_t):
    #学習データと教師データの分割
    df_fact = df_t['賃料']
    df_input = df_t.drop(['賃料'],axis=1)
    
    #予測値の生成
    df_input = xgb.DMatrix(df_input)
    pred = model.predict(df_input)
    
    #実値と予測値の表示
    df_pred = pd.DataFrame(pred,columns=['予測値'])
    df_final = pd.concat([df_fact,df_pred],axis=1)
    display(df_final)
    
    #モデルの性能を表示
    print('MAE:',mean_absolute_error(df_fact,df_pred))
    print('MSE:',mean_squared_error(df_fact,df_pred))
    print('RMSE:',np.sqrt(mean_squared_error(df_fact,df_pred)))


#RERDP(テストデータ)
RERDP(df_t)


In [ ]:
#目的変数とのクロス集計プログラム
def all_plot(df_final):
    import matplotlib.pyplot as plt
    import japanize_matplotlib
    #from sklearn.linear_model import LinearRegression
    
    y = df_final['賃料']
    columns = list(df_final.columns)
    
    fig, ax = plt.subplots(len(columns),1,figsize=(40,30*len(columns)))
    plt.rcParams["font.size"] = 30

    for i,column in enumerate(columns):
        x = df_final[column]
        
        ax[i].set_title(column)

        ax[i].scatter(x,y)
        
        print(column,'の相関係数：',x.corr(y))
        
    plt.show()

In [ ]:
all_plot(df_l)

In [ ]:
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns 
plt.figure(figsize=(2, 2))
sns.pairplot(df_l, size=2.0)

In [ ]:
#df_lの分散・共分散行列を見やすいヒートマップ形式にて出力
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns 
plt.rcParams["font.size"] = 10
sns.heatmap(df_l.corr(), annot=True, fmt='.2f')
#説明変数同士の相関は強くないため、多重共線性の心配はない事が分かる。

In [ ]:
#必要なライブラリをインポート
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder


#numpyのリスト表示制限を解除しておく
np.set_printoptions(threshold=np.inf)


#既に学習データとREDP関数を用いてAIが作成されているものとする
#model = RERL(df_l)


#住所と路線と間取りはラベルエンコーディングの都合により学習データ/テストデータにあったものしか使えない為、予め確保しておいた使える要素を表示させる
#上記の3つの要素はこの中から選んでもらう
#このプログラムは別のセルで起動させると見やすい
print(adressC,'\n','\n',stationC,'\n','\n',layoutC)


#(学習した範囲内の)任意のデータを入力して賃料を予測できる関数
#Real_Estate_Own_Data_Prediction
def REODP(address,station,access,mc_fees,k_fees,s_fees,area,layout,age):
    #入力したデータを辞書d_tryに格納する
    d_try = {
        '住所':address,
        '路線':station,
        '交通':access,
        '管理共益費':mc_fees,
        '礼金':k_fees,
        '敷金':s_fees,
        '専有面積':area,
        '間取り':layout,
        '築年数':age
    }
    
    #辞書d_tryをデータフレームdf_tryに変換する
    df_try = pd.DataFrame(d_try,index=['own'])
    
    #入力情報の確認用
    display(df_try)
    
    #ラベルエンコーディングを行い、文字列を数値化する
    df_try.住所 = LE1.transform(df_try.住所)
    df_try.路線 = LE2.transform(df_try.路線)
    df_try.間取り = LE3.transform(df_try.間取り)
    
    #データ型をfloat64で統一する
    df_try = df_try.astype('float64')
    
    #予測結果を表示する
    df_try = xgb.DMatrix(df_try)
    return print('予想賃料:',float(model.predict(df_try)),'万円')


#REODP(住所, 路線, 交通, 管理共益費, 礼金, 敷金, 専有面積, 間取り, 築年数)
#データ型に気をつける
#住所と路線と間取りはラベルエンコーディングの都合により学習データ/テストデータにあったものしか使えない為、上で表示させた要素から選ぶこと
REODP(address='神奈川県相模原市中央区淵野辺５'
      ,station='ＪＲ横浜線/相模原'
      ,access=15
      ,mc_fees=5000
      ,k_fees=0
      ,s_fees=0
      ,area=90
      ,layout='4LDK'
      ,age=19
     )

